In [15]:
from selenium import webdriver
import requests
from parsel import Selector

## 1) We obtain 10000 url of the properties with web driver.

In [12]:
driver = webdriver.Chrome(executable_path='chromedriver.exe')

# We add url of each house in 333 page in immoweb to houses_url list.
houses_url = []

for i in range(1, 334):
    # We used 'i' to build urls of the 333 page in immoweb.
    #   So, we can reach 333 pages with for loop.
    apikey = str(i)+'&orderBy=relevance'
    url = 'https://www.immoweb.be/en/search/house/for-sale?countries=BE&page='+apikey

    # An implicit wait tells WebDriver to poll the DOM for a
    #   certain amount of time when trying to find any element 
    #     (or elements) not immediately available. 
    driver.implicitly_wait(10)
    
    # The first thing you’ll want to do with WebDriver is navigate
    #   to a link. The normal way to do this is by calling get method:    
    driver.get(url)

    # Selector` allows you to select parts of an XML or HTML text using CSS
    #   or XPath expressions and extract data from it.
    sel = Selector(text=driver.page_source) 

    # xpath query of the houses in the immoweb page
    xpath_houeses = '//*[@id="main-content"]/li//h2//a/@href'
    
    # Find nodes matching the xpath ``query`` and return the result
    page_houses_url = sel.xpath(xpath_houeses).extract()
    
    # There are approximately 30 houses in each page.
    # We add each page url list to houses_url like matrix.
    houses_url.append(page_houses_url)

print(houses_url[2][4])
print(len(houses_url))

https://www.immoweb.be/en/classified/town-house/for-sale/strombeek-bever/1853/8911430?searchId=5f6c49339213c
333


## 2) We scrape all data of each house with requests

#### Each url represents a house and a house have many attributes like localty, type_property etc.  

In [69]:
import re
class HouseScraping:
    def __init__(self, url):
        self.url = url
        self.html = str(requests.get(self.url).content)
        self.sel = Selector(text=self.html)
        
        self.locality = self.locality()
        self.type_property = self.type_property()
        self.subtype = self.subtype()
        self.price = self.price()
        self.type_sale = self.type_sale()
        self.num_rooms = self.num_rooms()
        self.area = self.area()
        self.kitchen = self.kitchen()
        self.furnished = self.furnished()
        self.fire = self.fire()
        self.terrace_area = self.terrace_area()
        self.garden_area = self.garden_area()
        self.land = self.land()
        self.land_plot = self.land_plot()
        self.num_facade = self.num_facade()
        self.pool = self.pool()
        self.state = self.state()
    
    def locality(self):
        pass
    
    def type_property(self):
        pass
    
    def subtype(self):
        pass
    
    def price(self):
        pass
    
    def type_sale(self):
        pass
    
    def num_rooms(self):
        pass
    
    def area(self):
        pass
    
    def kitchen(self):
        pass
        
    def furnished(self):
        pass
    
    def fire(self):
        pass
    
    def terrace_area(self):
        pass
    
    def garden_area(self):
        pass
    
    def land(self):
        pass
    
    def land_plot(self):
        pass
    
    def num_facade(self):
        pass
    
    def pool(self):
        html_string = str(self.html)
        swim_regex = re.findall('swimming pool', html_doc)
        if swim_regex:
            return 1
        else:
            return 0
        
    def state(self):            
        pass

####       We collect all data in the houses_dict.

In [70]:
print(houses_url[0][0])

https://www.immoweb.be/en/classified/house/for-sale/woluwe-saint-pierre/1150/8856902?searchId=5f6c492536874


In [ ]:
from collections import defaultdict
houses_dict = defaultdict(list)

for page_list in houses_url:
    for url_a_house in page_list:
        
        houses_class = HouseScraping(url_a_house)
        
        houses_dict['Locality'].append(houses_class.locality)
        houses_dict['Type of property'].append(houses_class.type_property)
        houses_dict['Subtype of property'].append(houses_class.subtype)
        houses_dict['Price'].append(houses_class.price)
        houses_dict['Type of sale'].append(houses_class.type_sale)
        houses_dict['Number of rooms'].append(houses_class.num_rooms)
        houses_dict['Area'].append(houses_class.area)
        houses_dict['Fully equipped kitchen'].append(houses_class.kitchen)
        houses_dict['Furnished'].append(houses_class.furnished)
        houses_dict['Open fire'].append(houses_class.fire)
        houses_dict['Terrace'].append(houses_class.terrace_area)
        houses_dict['Garden'].append(houses_class.garden_area)
        houses_dict['Surface of the land'].append(houses_class.land)
        houses_dict['Surface area of the plot of land'].append(houses_class.land_plot)
        houses_dict['Number of facades'].append(houses_class.num_facade)
        houses_dict['Swimming pool'].append(houses_class.pool)
        houses_dict['State of the building'].append(houses_class.state)

## 3) We store all data to a csv file with dataframe.

In [ ]:
import pandas as pd
    
df = pd.DataFrame(houses_dict)
df.to_csv('all_data_of_the_houses.csv')